### Get access token
[Documentation](https://documentation.dataspace.copernicus.eu/APIs/Token.html)

### OAuth Setup
[Docs](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html#python), [Credentials](https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings)


In [1]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import json
import pathlib
from osgeo import gdal
from copernicus import Client, get_labels, get_sentinel2_product_ids, download_sentinel2_products
import building_dection as bd



c:\Users\tlaus\Documents\local\semester_thesis\sen2cor_analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
credentials = json.load(open('credentials.json'))
client = Client(credentials['username'], credentials['password'])

### Define Project Paths 

In [3]:
sn7_path = pathlib.Path('D:\snap_data\sn7')
data_path = sn7_path / 'data'
train_path = sn7_path / 'SN7_buildings_train' / 'train'
sentinel_path = sn7_path / 'sentinel-2'
vector_label_dir = sn7_path / 'vector_labels'

### Get Building Label Files

In [4]:
labels = get_labels(train_path)[:20]

# copy labels files to vector_label_dir
for label in labels:
    shutil.copy(label, vector_label_dir / label.name)

labels = [ vector_label_dir / label.name for label in labels]



### Init data dir and location dirs

In [ ]:
sn7_locations = []
for label in labels[:10]:
    label_name = str(label).split("_mosaic_")[-1].split("_Buildings")[0]
    loc_path = data_path / str(label).split("_mosaic_")[-1].split("_Buildings")[0].split("-")[1]
    sn7_loc = bd.SN7_Location(loc_path, label_name)
    sn7_loc.init_labels(label)
    sn7_locations.append(sn7_loc)

In [ ]:
sn7_locations

[Location(L15-0331E-1257N_1327_3160_13),
 Location(L15-0357E-1223N_1429_3296_13),
 Location(L15-0358E-1220N_1433_3310_13),
 Location(L15-0361E-1300N_1446_2989_13),
 Location(L15-0368E-1245N_1474_3210_13),
 Location(L15-0434E-1218N_1736_3318_13),
 Location(L15-0487E-1246N_1950_3207_13),
 Location(L15-0566E-1185N_2265_3451_13),
 Location(L15-0571E-1075N_2287_3888_13),
 Location(L15-0586E-1127N_2345_3680_13)]

### Get S2-L1C products

In [ ]:
for sn7_loc in sn7_locations:
    print(f"\nLoc: {sn7_loc.path.name}, Prod: {sn7_loc.product_name}, {sn7_loc.l1c_path}")
    try:
        sn7_loc.init_l1c(client, sentinel_path)
        print(f"\nLoc: {sn7_loc.path.name}, Prod: {sn7_loc.product_name}, {sn7_loc.l1c_path}")
    except Exception as e:
        print(e)
        print("\tNo L1C Product Found for: ", sn7_loc.label)
        # remove the location from the list
        sn7_locations.remove(sn7_loc)



Loc: 0331E, Prod: None, None
Product S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE already downloaded
label_extent: ulx: 615497.0	uly: 4206249.0	lrx: 619397.0	lry: 4202389.0
SAFE_extent: ulx:  600000	uly: 4300020	lrx: 709800	lry: 4190220

Loc: 0331E, Prod: S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE, D:\snap_data\sn7\data\0331E\S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE

Loc: 0357E, Prod: None, None
Product S2A_MSIL1C_20200109T183731_N0500_R027_T11SMS_20230428T041538.SAFE already downloaded
label_extent: ulx: 481127.0	uly: 3666556.0	lrx: 485247.0	lry: 3662476.0
SAFE_extent: ulx:  399960	uly: 3700020	lrx: 509760	lry: 3590220

Loc: 0357E, Prod: S2A_MSIL1C_20200109T183731_N0500_R027_T11SMS_20230428T041538.SAFE, D:\snap_data\sn7\data\0357E\S2A_MSIL1C_20200109T183731_N0500_R027_T11SMS_20230428T041538.SAFE

Loc: 0358E, Prod: None, None
Product S2A_MSIL1C_20200129T183621_N0208_R027_T11SNS_20200129T215547.SAFE already downloaded

In [ ]:
for sn7_loc in sn7_locations:
    print(f"Loc: {sn7_loc.path.name}, Prod: {sn7_loc.product_id}")
    if sn7_loc.product_id is None:
        print("\tNo L1C Product Found for: ", sn7_loc.label)
        # remove the location from the list
        sn7_locations.remove(sn7_loc)

Loc: 0331E, Prod: 085e9579-18af-4f27-b479-48cd70b8bb30
Loc: 0357E, Prod: 065a6272-73aa-4a33-9a21-b3e4b94e6517
Loc: 0361E, Prod: None
	No L1C Product Found for:  L15-0361E-1300N_1446_2989_13
Loc: 0434E, Prod: e66dd68b-1cf5-4361-9faf-999f4c7c94f0
Loc: 0487E, Prod: af75a99f-76d5-4b83-a07f-f04b938cffae
Loc: 0566E, Prod: 43aa7186-ffbb-4023-88fd-6530aabd08c3
Loc: 0571E, Prod: 9510aefd-a640-4b9a-9422-e3f589df20ea
Loc: 0586E, Prod: 984d45ac-16c6-5f2f-9a7f-dcfb93902375


In [ ]:
modifications = [
    {
        "flag": "SEN2COR_MODIFY_LUT_AEROSOL",
        "value": "MARITIME",
        "info": "Look Up Table Aerosol set to MARITIME",
        "name": "LUT_AEROSOL_MARITIME",
    },
    # {
    #     "flag": "SEN2COR_MODIFY_LUT_SEASON",
    #     "value": "WINTER",
    #     "info": "Look Up Table Mid Latitude Season set to WINTER",
    #     "name": "LUT_SEASON_WINTER",
    # },
]


In [ ]:
import sys
sys.path.append("../")
from l2a_analysis import L2A_Analysis

l2a_analysis = L2A_Analysis(report_name="sn7_cop", base_input_dir=sentinel_path, base_output_dir=sn7_path/"reports", resolution=10)
l2a_analysis.set_locations_sn7(sn7_locations[:56])
l2a_analysis.set_modifications(modifications)


Adding location 0331E, S2B_MSIL1C_20200110T185739_N0500_R113_T10SFH_20230426T213902.SAFE
Adding location 0357E, S2A_MSIL1C_20200109T183731_N0500_R027_T11SMS_20230428T041538.SAFE
Adding location 0368E, S2A_MSIL1C_20200106T182741_N0500_R127_T11SPA_20230426T210208.SAFE
Adding location 0434E, S2A_MSIL1C_20200111T173701_N0500_R055_T13SFR_20230428T081153.SAFE
Adding location 0487E, S2A_MSIL1C_20200112T170651_N0500_R069_T15SUA_20230629T201230.SAFE


In [ ]:
for k,v in l2a_analysis.locations.items():
    print(k, v['region_of_interest'])
len(l2a_analysis.locations)

0331E ['9564', '1740', '420', '420']
0357E ['3540', '8316', '432', '444']
0368E ['8940', '6144', '432', '432']
0434E ['2484', '2328', '348', '408']
0487E ['7836', '8460', '432', '432']


5

In [ ]:
lut_modifications = [
    {
        "flag": "SEN2COR_MODIFY_LUT_AEROSOL",
        "value": "MARITIME",
        "info": "Look Up Table Aerosol set to MARITIME",
        "name": "LUT_AEROSOL_MARITIME",
    },
    {
        "flag": "SEN2COR_MODIFY_LUT_SEASON",
        "value": "WINTER",
        "info": "Look Up Table Mid Latitude Season set to WINTER",
        "name": "LUT_SEASON_WINTER",
    },
    {
        "flag": "SEN2COR_MODIFY_LUT_OZONE",
        "value": "250",
        "info": "Look Up Table Ozone set to 250",
        "name": "LUT_OZONE_250",
    },
    {
        "flag": "SEN2COR_MODIFY_LUT_OZONE",
        "value": "331",
        "info": "Look Up Table Ozone set to 331 (standard mid latitude summer)",
        "name": "LUT_OZONE_331",
    },
    {
        "flag": "SEN2COR_MODIFY_LUT_OZONE",
        "value": "450",
        "info": "Look Up Table Ozone set to 450",
        "name": "LUT_OZONE_450",
    },
]

flag_modifications = [
    {
        "flag": "SEN2COR_MODIFY_FLAGS_WV",
        "value": "0",
        "info": "Turn off water vapour correction",
        "name": "FLAG_WV_OFF",
    },
    {
        "flag": "SEN2COR_MODIFY_FLAGS_CIRRUS",
        "value": "FALSE",
        "info": "Turn off cirrus correction",
        "name": "FLAG_CIRRUS_OFF",
    },
    {
        "flag": "SEN2COR_MODIFY_FLAGS_BRDF",
        "value": "1",
        "info": "Set BRDF correction to 1",
        "name": "FLAG_BRDF_1",
    },
    {
        "flag": "SEN2COR_MODIFY_FLAGS_BRDF",
        "value": "2",
        "info": "Set BRDF correction to 2",
        "name": "FLAG_BRDF_2",
    },
    {
        "flag": "SEN2COR_MODIFY_CALIB_VISIBILITY",
        "value": "20",
        "info": "Set visibility to 20km",
        "name": "CALIB_VISIBILITY_20",
    },
    {
        "flag": "SEN2COR_MODIFY_CALIB_VISIBILITY",
        "value": "80",
        "info": "Set visibility to 80km",
        "name": "CALIB_VISIBILITY_80",
    },
    {
        "flag": "SEN2COR_MODIFY_CALIB_VISIBILITY",
        "value": "120",
        "info": "Set visibility to 120km",
        "name": "CALIB_VISIBILITY_120",
    },
]

class_modifications = [
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "0",
        "info": "Set whole SCL layer to NO_DATA",
        "name": "class_NO_DATA",
    },
    # {
    #     'flag' : 'SEN2COR_MOD_const_class',
    #     'value' : '1',
    #     'info' : 'Set whole SCL layer to SATURATED_OR_DEFECTIVE',
    #     'name' : 'class_SATURATED_OR_DEFECTIVE'
    # },
    # {
    #     "flag": "SEN2COR_MOD_const_class",
    #     "value": "2",
    #     "info": "Set whole SCL layer to CASTED_SHADOWS",
    #     "name": "class_CASTED_SHADOWS",
    # },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "3",
        "info": "Set whole SCL layer to CLOUD_SHADOWS",
        "name": "class_CLOUD_SHADOWS",
    },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "4",
        "info": "Set whole SCL layer to VEGETATION",
        "name": "class_VEGETATION",
    },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "5",
        "info": "Set whole SCL layer to NOT_VEGETATED",
        "name": "class_NOT_VEGETATED",
    },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "6",
        "info": "Set whole SCL layer to WATER",
        "name": "class_WATER",
    },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "7",
        "info": "Set whole SCL layer to UNCLASSIFIED",
        "name": "class_UNCLASSIFIED",
    },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "8",
        "info": "Set whole SCL layer to CLOUD_MEDIUM_PROBABILITY",
        "name": "class_CLOUD_MEDIUM_PROBABILITY",
    },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "9",
        "info": "Set whole SCL layer to CLOUD_HIGH_PROBABILITY",
        "name": "class_CLOUD_HIGH_PROBABILITY",
    },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "10",
        "info": "Set whole SCL layer to THIN_CIRRUS",
        "name": "class_THIN_CIRRUS",
    },
    {
        "flag": "SEN2COR_MOD_const_class",
        "value": "11",
        "info": "Set whole SCL layer to SNOW",
        "name": "class_SNOW",
    },
]

total_modifications = lut_modifications + flag_modifications + class_modifications

23

In [ ]:
l2a_analysis.set_modifications(total_modifications)
l2a_analysis.run_sen2cor()
